# File paths and general generators

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import cv2
import os
import random
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/Users/imad/anaconda3/envs/dl_test_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
d1_base_path = "./COVIDvsARDS/main/"
d2_base_path = "./COVIDvsNORMAL/main/"
d3_base_path = "./COVIDvsVBPNEUMONIA/main/"

In [3]:
INIT_LR = 1e-3
EPOCHS = 25
BS = 8

In [4]:
def getImagesAndLabels(filepath, res):
    
    print("[INFO] loading images and labels...")
    imagePaths = list(paths.list_images(filepath))
    data = []
    labels = []

    # loop over the image paths
    for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]

        # load the image, swap color channels, and resize it to be a fixed
        # 224x224 pixels while ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (res, res))

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    # convert the data and labels to NumPy arrays while scaling the pixel
    # intensities to the range [0, 255]
    data = np.array(data) / 255.0
    labels = np.array(labels)

    # perform one-hot encoding on the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)
    
    print("Data: ", end="")
    print(data.shape)
    print("Labels: ", end="")
    print(labels.shape)
    
    return data, labels

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                rotation_range=20,
                                                                zoom_range=0.2,                 
                                                                horizontal_flip = True ,
                                                                fill_mode="nearest")

# VGG16 Model

In [21]:
vgg16_weightspath = "./weights/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

vgg16baseModel = tf.keras.applications.VGG16(weights=None, include_top=False,input_shape=(256,256,3))

vgg16baseModel.load_weights(vgg16_weightspath)

x = vgg16baseModel.output
x = tf.keras.layers.Conv2D(16, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(2, activation="softmax")(x)

for layer in vgg16baseModel.layers:
    layer.trainable = False

vgg16_model = tf.keras.Model(vgg16baseModel.input, x)
vgg16_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

vgg16_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [6]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = tf.keras.applications.VGG16(weights="imagenet", include_top=False,input_shape=(256,256,3))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = tf.keras.layers.AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = tf.keras.layers.Flatten(name="flatten")(headModel)
headModel = tf.keras.layers.Dense(64, activation="relu")(headModel)
headModel = tf.keras.layers.Dropout(0.5)(headModel)
headModel = tf.keras.layers.Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = tf.keras.optimizers.Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("DONE")

[INFO] compiling model...
DONE


# D2 normal

In [ ]:
# vgg16

In [7]:
d2_images, d2_labels = getImagesAndLabels(d2_base_path, 256)

[INFO] loading images and labels...
Data: (156, 256, 256, 3)
Labels: (156, 2)


In [8]:
(trainX, testX, trainY, testY) = train_test_split(d2_images, d2_labels,
	test_size=0.20, stratify=d2_labels, random_state=62)

In [9]:
trainX.shape

(124, 256, 256, 3)

In [10]:
testX.shape

(32, 256, 256, 3)

In [11]:
testY.shape

(32, 2)

In [13]:
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	train_datagen.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=10)

[INFO] training head...
  ...
    to  
  ['...']
Train for 15 steps, validate on 32 samples
Epoch 1/10
15/15 [==============================] - 69s 5s/step - loss: 0.8331 - accuracy: 0.4052 - val_loss: 0.7060 - val_accuracy: 0.5000
Epoch 2/10
15/15 [==============================] - 76s 5s/step - loss: 0.7417 - accuracy: 0.4741 - val_loss: 0.6893 - val_accuracy: 0.5625
Epoch 3/10
15/15 [==============================] - 71s 5s/step - loss: 0.7296 - accuracy: 0.4655 - val_loss: 0.6888 - val_accuracy: 0.5938
Epoch 4/10
15/15 [==============================] - 67s 4s/step - loss: 0.7232 - accuracy: 0.4483 - val_loss: 0.6922 - val_accuracy: 0.4688
Epoch 5/10
15/15 [==============================] - 72s 5s/step - loss: 0.7044 - accuracy: 0.4741 - val_loss: 0.6922 - val_accuracy: 0.5312
Epoch 6/10
15/15 [==============================] - 69s 5s/step - loss: 0.7027 - accuracy: 0.5250 - val_loss: 0.6908 - val_accuracy: 0.5000
Epoch 7/10
 2/15 [===>..........................] - ETA: 47s - loss:

KeyboardInterrupt: 

In [14]:
import gc
gc.collect()

859